In [1]:
import os
from pathlib import Path
TUTORIAL_DIR = Path(os.getcwd()).as_posix()

# Energytool Building

The aim of the tutorial is to introduce the energytool <code>Building</code> class.
It is build on top of [energyplus](https://energyplus.net/) and of the python library [eppy](https://github.com/santoshphilip/eppy).
It has been designed to simplify the HVAC, Domestic Hot Water (DHW) production and local energy production (Photovoltaic)

<img src="resources/building_class.png"  height="300">

- It has only one positional argument "idf_path". When instantiated it uses eppy.modeleditor <code>IDF</code> to read the idf file
- 7 <code>dict</code> are used to organise the HVAC DHW and production equipments objects. Additional object can be stored in the "other" <code>dict</code>, sucha as thermal comfort indicator objects.
- <code>property</code> are defined to compute specific results.

During simulation :
1- The <code>pre_process()</code> method is called. It calls the <code>pre_process()</code> method of each object contained in the previously described <code>dict</code>. Be careful, order matters in some cases
2- Energyplus is called and the simulation runs.
3- The <code>post_process()</code> method is called. It calls the <code>post_process()</code> method of each object contained in the previously described <code>dict</code>.

In [4]:

from energytool.building import Building

In [5]:
Building()

TypeError: __init__() missing 1 required positional argument: 'idf_path'